# 군집화(Clustering)
- 앞서 df_train_final 데이터 셋에 존재하는 연속형 변수들과 취급액(sales)의 상관관계를 확인해보았다.
- 하지만 판매량(pd_count)외에 상관관계가 높은 변수들은 존재하지 않았다.
    - 판매량의 경우 취급액과 높은 양의 상관관계가 있는 것이 당연함...
- 따라서 **군집화(Clustering)**를 통해 유의미한 패턴 및 요인들을 발견해보고자 한다!!

**<Raw Data 변수 설명>**
1. date: 방송일시
2. exp_mins: 노출(분)
3. mom_code: 마더코드
4. pd_code: 상품코드
5. pd_name: 상품명
6. pd_group: 상품군
7. pd_price: 판매단가
8. sales: 취급액
    - 취급액 = 판매단가 x 주문량
9. weekdays: 요일
10. seasons: 계절
    - 1: 겨울(winter): 12 ~ 2월
    - 2: 봄(spring): 3 ~ 5월
    - 3: 여름(summer): 6 ~ 8월
    - 4: 가을(autumn): 9 ~ 11월
11. rating: 시청률
    - 단위는 "%"
    - exp_mins(노출(분))을 기준으로 평균 값을 계산
12. temp: 기온
13. rain: 강수량
14. humidity: 습도
15. snow: 적설량
16. dust: 미세먼지 농도
17. is_rain: 비 내림 여부
    - 0: 비 내리지 않음
    - 1: 비 내림
18. is_snow: 눈 내림 여부
    - 0: 눈 내리지 않음
    - 1: 눈 내림

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
%matplotlib inline

# for문 진행 현황을 확인할 수 있는 패키지
from tqdm.notebook import tqdm

# 여러 개의 데이터 프레임을 한 번에 merge 해주기 위한 패키지
from functools import reduce

# 이 두 줄의 코드는 matplotlib의 기본 scheme말고, seaborn scheme을 세팅해준다
# 일일이 graph의 font size를 지정할 필요 없이, seaborn의 font_scale을 사용하면 편리하다
plt.style.use('seaborn')
sns.set(font_scale = 2.5)

# 그래프에서 한글 폰트 깨지는 문제를 해결해주기 위한 코드
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

import platform

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname = path).get_name()
    rc('font', family = font_name)
else:
    print('Unknown system... sorry~~~~')

# 데이터 셋 확인

In [2]:
df_train = pd.read_csv('C:/Users/82109/Desktop/Data Science/Competition/2020_Bigcontest_working/dataset/df_train_final2.csv')
df_train

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,...,snow,dust,is_rain,is_snow,pd_count,month,day,hour,month_cat,hour_cat
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,1,...,NaN,65.0,0,0,53,1,1,6,상반기,오전
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,1,...,NaN,65.0,0,0,110,1,1,6,상반기,오전
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,1,...,NaN,65.0,0,0,82,1,1,6,상반기,오전
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,1,...,NaN,65.0,0,0,175,1,1,6,상반기,오전
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,1,...,NaN,65.0,0,0,168,1,1,6,상반기,오전
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1,...,NaN,24.0,0,0,69,12,31,23,하반기,심야
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,1,...,NaN,27.0,0,0,287,1,1,0,상반기,심야
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,1,...,NaN,27.0,0,0,622,1,1,0,상반기,심야
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,1,...,NaN,27.0,0,0,88,1,1,0,상반기,심야


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37372 entries, 0 to 37371
Data columns (total 24 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       37372 non-null  object 
 1   exp_mins   37372 non-null  float64
 2   mom_code   37372 non-null  int64  
 3   pd_code    37372 non-null  int64  
 4   pd_name    37372 non-null  object 
 5   pd_group   37372 non-null  object 
 6   pd_price   37372 non-null  int64  
 7   sales      37372 non-null  float64
 8   weekdays   37372 non-null  object 
 9   seasons    37372 non-null  int64  
 10  rating     37372 non-null  float64
 11  temp       37372 non-null  float64
 12  rain       3489 non-null   float64
 13  humidity   37372 non-null  float64
 14  snow       368 non-null    float64
 15  dust       34952 non-null  float64
 16  is_rain    37372 non-null  int64  
 17  is_snow    37372 non-null  int64  
 18  pd_count   37372 non-null  int64  
 19  month      37372 non-null  int64  
 20  day   

# 결측치 처리
- rain, snow 변수는 사용하지 않을 것이므로 결측치 여부가 큰 문제가 되지 않는다.
- 하지만 dust 변수의 경우, 분석에 사용할 변수이므로 결측치 처리를 수행해주어야 한다.

In [4]:
df_train.isnull().sum()

date             0
exp_mins         0
mom_code         0
pd_code          0
pd_name          0
pd_group         0
pd_price         0
sales            0
weekdays         0
seasons          0
rating           0
temp             0
rain         33883
humidity         0
snow         37004
dust          2420
is_rain          0
is_snow          0
pd_count         0
month            0
day              0
hour             0
month_cat        0
hour_cat         0
dtype: int64

- 군집화를 하려면 dust 변수에 존재하는 2420개의 결측치들을 모두 처리해주어야 한다.
- 먼저 **"month-day-hour" 형태를 갖는 new_date 변수**를 새로 만들어주겠다.
    - new_date 변수를 기준으로 set()해서 중복 값들을 제거해주기 위함이다.

In [5]:
# 아래의 for문을 돌리기 위해, "0"값으로 채워진 new_date 변수를 생성
df_train['new_date'] = 0

for i in tqdm(range(len(df_train))):
    df_train['new_date'][i] = str(df_train['month'][i]) + '-' + str(df_train['day'][i]) + '-' + str(df_train['hour'][i])
df_train.head()

<ipython-input-5-af63bf274eab>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['new_date'][i] = str(df_train['month'][i]) + '-' + str(df_train['day'][i]) + '-' + str(df_train['hour'][i])
C:\Users\82109\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,...,dust,is_rain,is_snow,pd_count,month,day,hour,month_cat,hour_cat,new_date
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,1,...,65.0,0,0,53,1,1,6,상반기,오전,1-1-6
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,1,...,65.0,0,0,110,1,1,6,상반기,오전,1-1-6
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,1,...,65.0,0,0,82,1,1,6,상반기,오전,1-1-6
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,1,...,65.0,0,0,175,1,1,6,상반기,오전,1-1-6
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,1,...,65.0,0,0,168,1,1,6,상반기,오전,1-1-6


In [6]:
df_dust_null = df_train[df_train['dust'].isnull()]
dust_null_date = list(set(df_dust_null.new_date))
dust_null_date.sort()
dust_null_date

['1-4-9',
 '10-11-12',
 '10-11-13',
 '10-12-12',
 '10-13-10',
 '10-13-11',
 '10-13-12',
 '10-15-10',
 '10-15-11',
 '10-15-12',
 '10-15-13',
 '10-15-14',
 '10-15-18',
 '10-15-9',
 '10-16-12',
 '10-18-10',
 '10-18-11',
 '10-18-13',
 '10-18-18',
 '10-19-10',
 '10-19-11',
 '10-22-22',
 '10-25-10',
 '10-29-11',
 '10-3-1',
 '10-3-12',
 '10-3-14',
 '10-3-15',
 '10-3-7',
 '10-5-23',
 '10-6-0',
 '10-6-10',
 '10-6-11',
 '10-6-12',
 '10-6-14',
 '10-8-1',
 '10-8-10',
 '10-8-11',
 '10-8-12',
 '10-8-13',
 '10-8-14',
 '10-8-15',
 '10-8-16',
 '10-8-17',
 '10-8-18',
 '10-8-19',
 '10-8-20',
 '10-8-21',
 '10-8-22',
 '10-8-23',
 '10-8-6',
 '10-8-7',
 '10-8-8',
 '10-8-9',
 '10-9-0',
 '10-9-1',
 '10-9-10',
 '10-9-11',
 '10-9-12',
 '10-9-13',
 '10-9-14',
 '10-9-15',
 '10-9-16',
 '10-9-17',
 '10-9-18',
 '10-9-19',
 '10-9-20',
 '10-9-21',
 '10-9-22',
 '10-9-23',
 '10-9-6',
 '10-9-7',
 '10-9-8',
 '10-9-9',
 '11-1-18',
 '11-1-19',
 '11-1-20',
 '11-1-21',
 '11-1-22',
 '11-1-23',
 '11-10-20',
 '11-10-21',
 '11-10-

In [7]:
len(list(set(df_dust_null.new_date)))

489

- ```set()``` 함수를 사용하여 new_date 변수의 중복 값들을 제거해보니, 총 489개의 new_date가 존재한다.
- 이 489개의 일자들에 대한 결측값들은 다음과 같이 처리해주도록 하겠다.
    - 해당 시간대(hour)의 dust 변수가 결측치이면, 해당 일자(day)의 dust 평균 값으로 넣어주겠다.
        - ex) 2019년 1월 4일 9시에 미세먼지 농도(dust)가 결측치이면, 2019년 1월 4일 미세먼지 농도(dust)의 평균 값으로 대체
    - 만약 특정 일자(day)의 모든 시간대(hour)의 dust 변수가 결측치이면, dust 전체 평균 값으로 넣어주겠다.
        - ex) 2019년 9월 1일 모든 시간대에 미세먼지 농도(dust)가 결측치이면, 2019년 전체 미세먼지 농도(dsut)의 평균 값으로 대체

In [8]:
# 아래의 for문을 돌리기 위해, "0" 값들로 채워진 month_day 변수를 생성
df_train['month_day'] = 0

for i in tqdm(range(len(df_train))):
    df_train['month_day'][i] = str(df_train['month'][i]) + '/' + str(df_train['day'][i])
df_train.head()

<ipython-input-8-22884a4794ee>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['month_day'][i] = str(df_train['month'][i]) + '/' + str(df_train['day'][i])
C:\Users\82109\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,...,is_rain,is_snow,pd_count,month,day,hour,month_cat,hour_cat,new_date,month_day
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,1,...,0,0,53,1,1,6,상반기,오전,1-1-6,1/1
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,1,...,0,0,110,1,1,6,상반기,오전,1-1-6,1/1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,1,...,0,0,82,1,1,6,상반기,오전,1-1-6,1/1
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,1,...,0,0,175,1,1,6,상반기,오전,1-1-6,1/1
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,1,...,0,0,168,1,1,6,상반기,오전,1-1-6,1/1


- 해당 시간대(hour)의 dust 변수가 결측치이면, 아래와 같이 해당 일자(day)의 dust 평균 값으로 넣어주기 위해 **mean_dust 변수를 생성**해주겠다.
    - ex) 2019년 1월 4일 9시에 미세먼지 농도(dust)가 결측치이면, 2019년 1월 4일 미세먼지 농도(dust)의 평균 값으로 대체해주기 위해!!

In [9]:
g = df_train['dust'].groupby(df_train['month_day'])
df_mean_dust = pd.DataFrame(g.mean()).reset_index()
df_mean_dust.rename(columns = {'dust':'mean_dust'}, inplace = True)

df_mean_dust['mean_dust'] = round(df_mean_dust['mean_dust']) # 정수로 반올림
df_mean_dust

,month_day,mean_dust
0,1/1,39.0
1,1/10,62.0
2,1/11,86.0
3,1/12,112.0
4,1/13,133.0
...,...,...
360,9/5,10.0
361,9/6,19.0
362,9/7,9.0
363,9/8,34.0


- 아래와 같이 특정 일자(day)의 모든 시간대(hour)의 dust 변수가 결측치인 경우, dust 전체 평균 값으로 넣어주겠다.
    - ex) 2019년 9월 1일 모든 시간대에 미세먼지 농도(dust)가 결측치이면, 2019년 전체 미세먼지 농도(dust)의 평균 값으로 대체

In [10]:
df_mean_dust[df_mean_dust['mean_dust'].isnull()]

,month_day,mean_dust
61,10/9,NaN
325,8/29,NaN
327,8/30,NaN
328,8/31,NaN
335,9/1,NaN
349,9/22,NaN


In [11]:
# 2019년 전체 미세먼지 농도(dust)의 (정수로 반올림한) 평균 값으로 대체
df_mean_dust['mean_dust'] = df_mean_dust['mean_dust'].fillna(round(np.mean(df_train['dust'])))
# 결측치들이 잘 채워졌는지 확인
df_mean_dust[df_mean_dust['mean_dust'].isnull()]

,month_day,mean_dust


In [12]:
df_mean_dust

,month_day,mean_dust
0,1/1,39.0
1,1/10,62.0
2,1/11,86.0
3,1/12,112.0
4,1/13,133.0
...,...,...
360,9/5,10.0
361,9/6,19.0
362,9/7,9.0
363,9/8,34.0


- 기존의 데이터 셋(df_train)에 새로 만든 df_mean_dust 데이터 셋을 결합해주겠다.
    - **month_day 변수를 조인키**로 **inner join** 해주겠다.

In [13]:
df_train2 = pd.merge(df_train, df_mean_dust, how = 'inner', on = 'month_day')
df_train2

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,...,is_snow,pd_count,month,day,hour,month_cat,hour_cat,new_date,month_day,mean_dust
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,1,...,0,53,1,1,6,상반기,오전,1-1-6,1/1,39.0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,1,...,0,110,1,1,6,상반기,오전,1-1-6,1/1,39.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,1,...,0,82,1,1,6,상반기,오전,1-1-6,1/1,39.0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,1,...,0,175,1,1,6,상반기,오전,1-1-6,1/1,39.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,1,...,0,168,1,1,6,상반기,오전,1-1-6,1/1,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:20:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,1664000.0,Tuesday,1,...,0,12,12,31,23,하반기,심야,12-31-23,12/31,28.0
37368,2019-12-31 23:40:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,9149000.0,Tuesday,1,...,0,52,12,31,23,하반기,심야,12-31-23,12/31,28.0
37369,2019-12-31 23:40:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,15282000.0,Tuesday,1,...,0,91,12,31,23,하반기,심야,12-31-23,12/31,28.0
37370,2019-12-31 23:40:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,2328000.0,Tuesday,1,...,0,15,12,31,23,하반기,심야,12-31-23,12/31,28.0


- 이제 기존의 dust 변수가 결측값이면, 새로 만든 mean_dust 변수 값으로 대체해주도록 하겠다.

In [14]:
df_train2['dust'] = df_train2['dust'].fillna(df_train2['mean_dust'])
df_train2['dust'].isnull().sum()

0

- 마지막으로 dust 변수의 결측값 처리를 위해 만들어 준 변수들은 제거해주도록 하겠다.
    - **new_date, month_day, mean_dust 변수 제거**

In [15]:
df_train_fill_dust = df_train2.drop(['new_date', 'month_day', 'mean_dust'], axis = 1, inplace = False)
df_train_fill_dust.head()

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,...,snow,dust,is_rain,is_snow,pd_count,month,day,hour,month_cat,hour_cat
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,1,...,NaN,65.0,0,0,53,1,1,6,상반기,오전
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,1,...,NaN,65.0,0,0,110,1,1,6,상반기,오전
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,1,...,NaN,65.0,0,0,82,1,1,6,상반기,오전
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,1,...,NaN,65.0,0,0,175,1,1,6,상반기,오전
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,1,...,NaN,65.0,0,0,168,1,1,6,상반기,오전


- 여기까지 해서 dust 변수의 결측값들을 모두 채워주었다!!

# 데이터 타입 변경
- 분석에 사용할 컬럼들 중, 범주형 변수들의 데이터 타입을 'category'로 변경해주겠다.

In [16]:
categoricalFeatureNames = ['month', 'day', 'hour', 'month_cat', 'hour_cat', 'weekdays', 'seasons',
                           'pd_group', 'is_rain', 'is_snow']

for var in categoricalFeatureNames:
    df_train_fill_dust[var] = df_train_fill_dust[var].astype('category')
df_train_fill_dust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37372 entries, 0 to 37371
Data columns (total 24 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   date       37372 non-null  object  
 1   exp_mins   37372 non-null  float64 
 2   mom_code   37372 non-null  int64   
 3   pd_code    37372 non-null  int64   
 4   pd_name    37372 non-null  object  
 5   pd_group   37372 non-null  category
 6   pd_price   37372 non-null  int64   
 7   sales      37372 non-null  float64 
 8   weekdays   37372 non-null  category
 9   seasons    37372 non-null  category
 10  rating     37372 non-null  float64 
 11  temp       37372 non-null  float64 
 12  rain       3489 non-null   float64 
 13  humidity   37372 non-null  float64 
 14  snow       368 non-null    float64 
 15  dust       37372 non-null  float64 
 16  is_rain    37372 non-null  category
 17  is_snow    37372 non-null  category
 18  pd_count   37372 non-null  int64   
 19  month      37372 non-null

# 범주형 변수 추출
- K-prototype 알고리즘을 사용할 것이므로, One-Hot Encoding 작업은 수행해주지 않겠다.

In [17]:
df_cate = df_train_fill_dust[['month', 'day', 'hour', 'month_cat', 'hour_cat', 'weekdays', 
                              'seasons', 'pd_group', 'is_rain', 'is_snow']]
df_cate.head()

,month,day,hour,month_cat,hour_cat,weekdays,seasons,pd_group,is_rain,is_snow
0,1,1,6,상반기,오전,Tuesday,1,의류,0,0
1,1,1,6,상반기,오전,Tuesday,1,의류,0,0
2,1,1,6,상반기,오전,Tuesday,1,의류,0,0
3,1,1,6,상반기,오전,Tuesday,1,의류,0,0
4,1,1,6,상반기,오전,Tuesday,1,의류,0,0


# 연속형 변수 추출 및 Feature Scaling
- 스케일링 기법으로는 ```MinMaxScaler()```를 사용할 것이다.

In [18]:
# 연속형 변수만 추출된 데이터 셋
df_cont = df_train_fill_dust[['exp_mins', 'pd_price', 'rating', 'temp', 'humidity', 'dust']]

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(df_cont)
df_cont_scaled = scaler.transform(df_cont)

# transform()을 수행하면 스케일 변환된 데이터 셋이 Numpy ndarray로 반환되기 때문에, 이를 DataFrame으로 변경
df_cont_scaled = pd.DataFrame(df_cont_scaled, columns = df_cont.columns)

df_cont_scaled.head()

,exp_mins,pd_price,rating,temp,humidity,dust
0,0.46714,0.003423,0.0,0.061053,0.574713,0.237736
1,0.46714,0.003423,0.0,0.061053,0.574713,0.237736
2,0.46714,0.003423,0.0,0.061053,0.574713,0.237736
3,0.46714,0.003423,0.0,0.061053,0.574713,0.237736
4,0.46714,0.003423,0.0,0.061053,0.574713,0.237736


# 군집화를 위한 데이터 셋 생성
- "범주형 변수" + "스케일링 처리된 연속형 변수"

In [19]:
df_cate.shape

(37372, 10)

In [20]:
df_cont_scaled.shape

(37372, 6)

In [21]:
df_cluster = pd.concat([df_cate, df_cont_scaled], axis = 1)
df_cluster

,month,day,hour,month_cat,hour_cat,weekdays,seasons,pd_group,is_rain,is_snow,exp_mins,pd_price,rating,temp,humidity,dust
0,1,1,6,상반기,오전,Tuesday,1,의류,0,0,0.46714,0.003423,0.000000,0.061053,0.574713,0.237736
1,1,1,6,상반기,오전,Tuesday,1,의류,0,0,0.46714,0.003423,0.000000,0.061053,0.574713,0.237736
2,1,1,6,상반기,오전,Tuesday,1,의류,0,0,0.46714,0.003423,0.000000,0.061053,0.574713,0.237736
3,1,1,6,상반기,오전,Tuesday,1,의류,0,0,0.46714,0.003423,0.000000,0.061053,0.574713,0.237736
4,1,1,6,상반기,오전,Tuesday,1,의류,0,0,0.46714,0.003423,0.000000,0.061053,0.574713,0.237736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,12,31,23,하반기,심야,Tuesday,1,주방,0,0,0.46714,0.017077,0.000000,0.071579,0.402299,0.083019
37368,12,31,23,하반기,심야,Tuesday,1,주방,0,0,0.46714,0.020866,0.172264,0.071579,0.402299,0.083019
37369,12,31,23,하반기,심야,Tuesday,1,주방,0,0,0.46714,0.019603,0.172264,0.071579,0.402299,0.083019
37370,12,31,23,하반기,심야,Tuesday,1,주방,0,0,0.46714,0.018340,0.172264,0.071579,0.402299,0.083019


# 군집화(Clustering)

## K-prototype 알고리즘
- 우리가 **사용할 데이터 셋은 범주형 변수와 연속형 변수를 모두 포함**하고 있다.
- 따라서 **K-prototype 알고리즘**을 사용하도록 하겠다.

- 먼저 분석에 사용할 데이터 셋을 ndarray 형태로 변환해주겠다.

In [23]:
# Converting the dataset into ndarray
df_cluster_ndarray = df_cluster.to_numpy()
df_cluster_ndarray

array([[1, 1, 6, ..., 0.06105263157894736, 0.574712643678161,
        0.23773584905660378],
       [1, 1, 6, ..., 0.06105263157894736, 0.574712643678161,
        0.23773584905660378],
       [1, 1, 6, ..., 0.06105263157894736, 0.574712643678161,
        0.23773584905660378],
       ...,
       [12, 31, 23, ..., 0.07157894736842105, 0.4022988505747127,
        0.08301886792452831],
       [12, 31, 23, ..., 0.07157894736842105, 0.4022988505747127,
        0.08301886792452831],
       [12, 31, 23, ..., 0.07157894736842105, 0.4022988505747127,
        0.08301886792452831]], dtype=object)

### 군집화를 위한 최적의 K 값 찾기

In [24]:
from kmodes.kprototypes import KPrototypes

cost = []
for num_clusters in tqdm(list(range(2, 6))):
    kproto = KPrototypes(n_clusters = num_clusters, init = 'Huang', n_init = 50, max_iter = 15, n_jobs = -2, random_state = 0) 
    kproto.fit_predict(df_cluster_ndarray, categorical = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    cost.append(kproto.cost_)

plt.plot(cost)
plt.xlabel('K')
plt.ylabel('cost')
plt.show()

KeyboardInterrupt: 

- Elbow 방법으로 찾은 최적의 K 값은 ???라고 나온다.

### 최적의 K 값으로 군집화 수행
- **Elbow 방법**을 통해 **최적의 K 값은 ???라고 판단**하였다.

In [ ]:
kproto = KPrototypes(n_clusters = ?, init = 'Huang', n_init = 50, max_iter = 15, n_jobs = -2, random_state = 0) 
clusters = kproto.fit_predict(df_cluster_ndarray, categorical = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
cluster_dict = []
for c in clusters:
    cluster_dict.append(c)

# 각 데이터가 속하는 군집을 의미하는 'cluster' 변수를 df_train_fill_dust 데이터 셋에 추가
df_train_fill_dust['cluster'] = cluster_dict
df_train_fill_dust.cluster.value_counts()